In [1]:
## load packages
import pandas as pd
import FlowCytometryTools as fct
import numpy as np
import pandas as pd
import os
import sklearn.metrics as skmet
import itertools

from pylab import *
from plotnine import *
from plotnine.data import *
from itertools import combinations

# make ggplot classic looking
ggplot_classic = theme(panel_background = element_rect(fill='white'), panel_border=element_rect(color='black', size=1), panel_grid_major = element_blank(), panel_grid_minor = element_blank(), legend_box = element_rect(fill="white", size = 0.5)) 

C:\Users\nickh\Anaconda3\lib\site-packages\FlowCytometryTools\core\docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)
C:\Users\nickh\Anaconda3\lib\site-packages\FlowCytometryTools\core\docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)


In [2]:
## define function to remove quotes from column
rm_quote = lambda x: x.replace('"', '')

## load dataset
df = pd.read_csv('../data/post-sort.LOG', names = ['image_compartment', 'date_time', 'time_elapsed', 'well_site', 'wavelength', 'obj_number', 'area', 'intensity'], converters={'image_compartment': rm_quote, 'date_time': rm_quote, 'time_elapsed': rm_quote, 'well_site': rm_quote, 'obj_number': rm_quote})

## remove duplicated rows
df = df.drop_duplicates()

df.iloc[:10, :]

,image_compartment,date_time,time_elapsed,well_site,wavelength,obj_number,area,intensity
0,NIR_and_cytoplasm,11:04:30.614,00:00:00.000,A02 : Site 10,700,1,79.8768,13957.9
1,NIR_and_nucleus,11:04:29.810,00:00:00.000,A02 : Site 10,520,1,101.5100,12503.3
2,NIR_and_cytoplasm,11:04:30.614,00:00:00.000,A02 : Site 10,700,2,128.1360,13290.7
3,NIR_and_nucleus,11:04:29.810,00:00:00.000,A02 : Site 10,520,2,299.9540,12455.0
4,NIR_and_cytoplasm,11:04:30.614,00:00:00.000,A02 : Site 10,700,3,88.6133,16945.5
5,NIR_and_nucleus,11:04:29.810,00:00:00.000,A02 : Site 10,520,3,139.3680,30320.5
6,NIR_and_cytoplasm,11:04:30.614,00:00:00.000,A02 : Site 10,700,4,148.1050,13533.6
7,NIR_and_nucleus,11:04:29.810,00:00:00.000,A02 : Site 10,520,4,358.1980,26741.7
8,NIR_and_cytoplasm,11:04:30.614,00:00:00.000,A02 : Site 10,700,5,203.8520,13289.7
9,NIR_and_nucleus,11:04:29.810,00:00:00.000,A02 : Site 10,520,5,386.9030,19552.4


In [3]:
# melt the dataframe such that the id_vars are well_site and obj_number
df2 = pd.melt(df, id_vars = ['well_site', 'obj_number', 'image_compartment'], value_vars = ['area', 'intensity'])

df2.iloc[:10,:]

,well_site,obj_number,image_compartment,variable,value
0,A02 : Site 10,1,NIR_and_cytoplasm,area,79.8768
1,A02 : Site 10,1,NIR_and_nucleus,area,101.5100
2,A02 : Site 10,2,NIR_and_cytoplasm,area,128.1360
3,A02 : Site 10,2,NIR_and_nucleus,area,299.9540
4,A02 : Site 10,3,NIR_and_cytoplasm,area,88.6133
5,A02 : Site 10,3,NIR_and_nucleus,area,139.3680
6,A02 : Site 10,4,NIR_and_cytoplasm,area,148.1050
7,A02 : Site 10,4,NIR_and_nucleus,area,358.1980
8,A02 : Site 10,5,NIR_and_cytoplasm,area,203.8520
9,A02 : Site 10,5,NIR_and_nucleus,area,386.9030


In [4]:
# use a pivot table to reorganize data
df3 = df2.pivot_table(index = ['well_site', 'obj_number'], columns = ['image_compartment', 'variable'], values = 'value')

# add nc_ratio to the new datatable
df3['', 'nc_ratio'] = np.log(df3['NIR_and_nucleus', 'intensity'] / df3['NIR_and_cytoplasm', 'intensity'])

df3.iloc[:10,:]

image_compartment        NIR_and_cytoplasm           NIR_and_nucleus  \
variable                              area intensity            area   
well_site     obj_number                                               
 A02 : Site 1  1                   216.333   13593.6         500.478   
               10                  178.059   11705.8         305.362   
               11                  195.532   24359.2         412.697   
               12                  241.295   13110.8         596.996   
               13                  220.077   13228.4         470.940   
               14                  207.596   13838.3         450.971   
               15                  203.436   12923.9         376.919   
               16                  176.395   12500.9         300.786   
               17                  285.809   14301.0         893.206   
               18                  208.845   13358.9         456.795   

image_compartment                             
variable                 intensity  nc_ratio  
well_site     obj_number                      
 A02 : Site 1  1           18470.5  0.306576  
               10          11853.4  0.012530  
               11          33781.6  0.327007  
               12          17865.0  0.309407  
               13          16416.6  0.215927  
               14          16532.7  0.177900  
               15          16984.3  0.273211  
               16          19895.5  0.464693  
               17          17921.9  0.225694  
               18          16577.5  0.215864

In [5]:
# separate well and site with index
well_site = pd.Series(df3.index.get_level_values(0))

# split wells and sites
wells = well_site.apply(lambda x: x.split(' : ')[0].strip())
sites = well_site.apply(lambda x: x.split(' : ')[1].strip())

# remake the index using wells, sites, and obj_number
df4 = df3.set_index([wells, sites, df3.index.get_level_values(1)])
df4.index.names = ['well', 'site', 'obj_number']

df4.iloc[:10,:]

image_compartment      NIR_and_cytoplasm           NIR_and_nucleus            \
variable                            area intensity            area intensity   
well site   obj_number                                                         
A02  Site 1  1                   216.333   13593.6         500.478   18470.5   
             10                  178.059   11705.8         305.362   11853.4   
             11                  195.532   24359.2         412.697   33781.6   
             12                  241.295   13110.8         596.996   17865.0   
             13                  220.077   13228.4         470.940   16416.6   
             14                  207.596   13838.3         450.971   16532.7   
             15                  203.436   12923.9         376.919   16984.3   
             16                  176.395   12500.9         300.786   19895.5   
             17                  285.809   14301.0         893.206   17921.9   
             18                  208.845   13358.9         456.795   16577.5   

image_compartment                 
variable                nc_ratio  
well site   obj_number            
A02  Site 1  1          0.306576  
             10         0.012530  
             11         0.327007  
             12         0.309407  
             13         0.215927  
             14         0.177900  
             15         0.273211  
             16         0.464693  
             17         0.225694  
             18         0.215864

In [6]:
# load well map and merge with this
well_map = pd.read_csv('../data/post-sort-metadat.csv')

well_map

,well,replicate,bin
0,A01,R2T1,0ms
1,A02,R2T1,50ms
2,A03,R2T1,200ms
3,A04,R2T1,800ms
4,A06,R1T2,0ms
5,B06,R1T2,50ms
6,C01,R1T1,0ms
7,C02,R1T1,50ms
8,C03,R1T1,200ms
9,C04,R1T1,800ms


In [7]:
# merge dataframes
df5 = df4.merge(well_map, left_on = df4.index.get_level_values(0), right_on = 'well')

# rename some columns
df5.columns = ['NIR_cyt_area', 'NIR_cyt_int', 'NIR_nuc_area', 'NIR_nuc_int', 'nc_ratio', 'well', 'replicate', 'bin']

# categorize bin, with order
bins_cat = pd.api.types.CategoricalDtype(categories=["0ms", "50ms", "200ms", "800ms"], ordered = True)
df5.bin = df5.bin.astype(bins_cat)

# write to csv R1T2 and R2T2, which both have all 4 bins imaged
df5.loc[((df5.replicate == 'R1T2') | (df5.replicate == 'R2T2')), :].to_csv('NLS_post_sort_imaging.csv', index = False)


df5.iloc[:10,:]

C:\Users\nickh\AppData\Roaming\Python\Python37\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,NIR_cyt_area,NIR_cyt_int,NIR_nuc_area,NIR_nuc_int,nc_ratio,well,replicate,bin
0,216.333,13593.6,500.478,18470.5,0.306576,A02,R2T1,50ms
1,178.059,11705.8,305.362,11853.4,0.012530,A02,R2T1,50ms
2,195.532,24359.2,412.697,33781.6,0.327007,A02,R2T1,50ms
3,241.295,13110.8,596.996,17865.0,0.309407,A02,R2T1,50ms
4,220.077,13228.4,470.940,16416.6,0.215927,A02,R2T1,50ms
5,207.596,13838.3,450.971,16532.7,0.177900,A02,R2T1,50ms
6,203.436,12923.9,376.919,16984.3,0.273211,A02,R2T1,50ms
7,176.395,12500.9,300.786,19895.5,0.464693,A02,R2T1,50ms
8,285.809,14301.0,893.206,17921.9,0.225694,A02,R2T1,50ms
9,208.845,13358.9,456.795,16577.5,0.215864,A02,R2T1,50ms


In [8]:
# use data from technical replicates for which there are all four bins imaged
df6 = df5.loc[((df5.replicate == 'R1T2') | (df5.replicate == 'R2T2')),:]

df6.iloc[:10,]

,NIR_cyt_area,NIR_cyt_int,NIR_nuc_area,NIR_nuc_int,nc_ratio,well,replicate,bin
4123,110.2470,13595.3,169.7380,13834.6,0.017449,A06,R1T2,0ms
4124,133.9600,12472.5,153.5130,11944.8,-0.043230,A06,R1T2,0ms
4125,131.8800,14836.3,205.1000,13827.9,-0.070389,A06,R1T2,0ms
4126,147.2730,15008.3,198.0280,14080.0,-0.063848,A06,R1T2,0ms
4127,90.2774,15956.2,74.8845,22108.4,0.326110,A06,R1T2,0ms
4128,151.4330,13134.3,350.7090,12643.1,-0.038116,A06,R1T2,0ms
4129,148.1050,12659.7,220.0770,12429.5,-0.018351,A06,R1T2,0ms
4130,84.8691,16207.7,74.0525,22307.1,0.319419,A06,R1T2,0ms
4131,142.6970,12730.7,207.1800,12538.8,-0.015189,A06,R1T2,0ms
4132,148.9370,14395.1,216.3330,13563.9,-0.059476,A06,R1T2,0ms


In [9]:
# plot the results
s2b = ggplot(aes(x = 'bin', y = 'nc_ratio', fill = 'bin'), data = df6) 
s2b_boxp = geom_boxplot(outlier_alpha = 0.05)
s2b_them = theme_classic() + theme(axis_text=element_text(size=12),
                                   axis_text_x=element_text(angle = 45, hjust = 1),
                                   axis_title=element_text(size=14),
                                   strip_text = element_text(size=14),
                                   strip_background = element_blank(),
                                  text = element_text(family = 'Arial'))
s2b_ylab = ylab('Log$_{2}$(N:C ratio)')
s2b_xlab = xlab('Activation bin')
s2b_scfi = scale_fill_manual(values = ['#ffffff', '#fee6ce', '#fd8d3c', '#a63603'])
s2b_face = facet_wrap('~replicate', ncol = 2)
s2b_guid = guides(fill=False)

s2bp = s2b + s2b_boxp + s2b_them + s2b_ylab + s2b_xlab + s2b_scfi + s2b_face + s2b_guid

ggsave(s2bp, 'SF2B.png', dpi = 900, width = 3, height = 2)

C:\Users\nickh\AppData\Roaming\Python\Python37\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 3 x 2 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\nickh\AppData\Roaming\Python\Python37\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: SF2B.png
  warn('Filename: {}'.format(filename), PlotnineWarning)


In [10]:
df6.groupby(['well','replicate','bin']).groups


{('A06',
  'R1T2',
  '0ms'): Int64Index([4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132,
             ...
             8503, 8504, 8505, 8506, 8507, 8508, 8509, 8510, 8511, 8512],
            dtype='int64', length=4390),
 ('B06',
  'R1T2',
  '50ms'): Int64Index([8513, 8514, 8515, 8516, 8517, 8518, 8519, 8520, 8521, 8522,
             ...
             9805, 9806, 9807, 9808, 9809, 9810, 9811, 9812, 9813, 9814],
            dtype='int64', length=1302),
 ('C06',
  'R1T2',
  '200ms'): Int64Index([17127, 17128, 17129, 17130, 17131, 17132, 17133, 17134, 17135,
             17136,
             ...
             18421, 18422, 18423, 18424, 18425, 18426, 18427, 18428, 18429,
             18430],
            dtype='int64', length=1304),
 ('D01',
  'R2T2',
  '0ms'): Int64Index([18431, 18432, 18433, 18434, 18435, 18436, 18437, 18438, 18439,
             18440,
             ...
             25336, 25337, 25338, 25339, 25340, 25341, 25342, 25343, 25344,
             25345],
            dt